In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import json
import folium
import warnings


In [2]:
# 필요한 정보

# 주유소명 : name_list
# 주소 : address_list
# 상표 : brand_list
# 보통휘발유 : gasolin_list
# 경유 : disel_list
# 세차장 : cwsh_yn_list
# 충전소 : lpg_yn_list
# 경정비 : maint_yn_list
# 편의점 : cvs_yn_list
# 24시영업 : sel24_yn_list

name_list = []
address_list = []
brand_list = []
gasolin_list = []
disel_list = []
cwsh_yn_list = []
lpg_yn_list = []
maint_yn_list = []
cvs_yn_list = []
sel24_yn_list = []

In [3]:
options = webdriver.ChromeOptions()

prefs = {
    'download.default_directory':'/home/haneol/dev_ws/EDA/data/',
    'download.propt_for_download':False
}

options.add_experimental_option('prefs', prefs)

url = 'https://www.opinet.co.kr/searRgSelect.do'
driver = webdriver.Chrome(
    service=Service('../driver/chromedriver-linux64/chromedriver'),
    options=options
)
driver.get(url)

In [4]:
# 시도 목록 가져오기
sido = driver.find_element(By.ID, 'SIDO_NM0')
sido_list = sido.find_elements(By.TAG_NAME, 'option')
sido_names = [sido_name.get_attribute('value') for sido_name in sido_list]

# 시군구 정보 가져오기
gu = driver.find_element(By.ID, 'SIGUNGU_NM0')
gu_list = gu.find_elements(By.TAG_NAME, 'option')
gu_names = [gu_name.get_attribute('value') for gu_name in gu_list]

# 불필요한 값 제거
sido_names = sido_names[1:]
gu_names.remove('')
print(len(gu_names))

25


In [5]:
for gu_name in tqdm(gu_names):
    gu = driver.find_element(By.ID, 'SIGUNGU_NM0')
    gu.send_keys(gu_name)

    search_result = driver.find_element(By.ID, 'body1')
    station_list = search_result.find_elements(By.TAG_NAME, 'tr')

    for each_station in station_list:
        click_tag = each_station.find_element(By.TAG_NAME, 'a')
        click_tag.click()
        detail_info = driver.find_element(By.CLASS_NAME, 'ollehmap-info-content-overflow.ollehmap-event')
        
        name = ''
        panup = driver.find_element(By.CLASS_NAME, 'control-panup.ollehmap-control-clickable.ollehmap-event')
        while name == '':
            actions = ActionChains(driver)
            actions.click(panup)
            actions.perform()
            name = detail_info.find_element(By.ID, 'os_nm').text

        address = detail_info.find_element(By.ID, 'rd_addr').text
        brand = detail_info.find_element(By.ID, 'poll_div_nm').text
        gasolin = detail_info.find_element(By.ID, 'b027_p').text
        disel = detail_info.find_element(By.ID, 'd047_p').text
        cwsh_yn = detail_info.find_element(By.ID, 'cwsh_yn').get_attribute('src')
        lpg_yn = detail_info.find_element(By.ID, 'lpg_yn').get_attribute('src')
        maint_yn = detail_info.find_element(By.ID, 'maint_yn').get_attribute('src')
        cvs_yn = detail_info.find_element(By.ID, 'cvs_yn').get_attribute('src')
        sel24_yn = detail_info.find_element(By.ID, 'sel24_yn').get_attribute('src')
        
        name_list.append(name)
        address_list.append(address)
        brand_list.append(brand)
        gasolin_list.append(gasolin)
        disel_list.append(disel)
        cwsh_yn_list.append(cwsh_yn)
        lpg_yn_list.append(lpg_yn)
        maint_yn_list.append(maint_yn)
        cvs_yn_list.append(cvs_yn)
        sel24_yn_list.append(sel24_yn)

print(name_list)
print(address_list)
print(brand_list)
print(gasolin_list)
print(disel_list)
print(cwsh_yn_list)
print(lpg_yn_list)
print(maint_yn_list)
print(cvs_yn_list)
print(sel24_yn_list)

  0%|          | 0/25 [00:00<?, ?it/s]

['(주)보성 세곡주유소', 'HD현대오일뱅크㈜직영 산성셀프주유소', '방죽주유소', '자곡셀프주유소', 'HD현대오일뱅크㈜직영 도곡셀프주유소', '오일프러스 셀프', 'HD현대오일뱅크㈜직영 유진주유소', 'SK서광주유소', '극동유화㈜ 개나리주유소', '(주)선진도곡주유소', '(주)중앙에너비스 수서지점', '일원주유소', '지에스칼텍스㈜에너지플러스허브GS타워', '대교셀프주유소', '삼성동주유소', 'SK에너지㈜ 진달래주유소', 'SK에너지(주) 매봉주유소', '(유)동하석유 힐탑셀프주유소', '대동석유 압구정주유소', '에스제이에너지산업(주) 개포주유소', 'HD현대오일뱅크㈜직영 신사현대주유소', '지에스칼텍스㈜ 은마주유소', 'HD현대오일뱅크㈜직영 한양주유소', '지에스칼텍스(주)학여울주유소', '지에스칼텍스㈜ 삼성로주유소', '대성석유(주)동호주유소', '대청주유소', '명품주유소', '(주)소모 쎈트럴주유소', '㈜새서울석유 새서울주유소', '갤러리아주유소', 'SK논현주유소', '(주)제이제이네트웍스 제이제이주유소', '(주)만정에너지 삼보주유소', '재건에너지 재정제2주유소 고속셀프지점', '구천면주유소', '방아다리주유소', '(주)삼표에너지 고덕주유소', '주)지유에너지직영 오렌지주유소', '명일주유소', '(주)소모 신월주유소', '(주)퍼스트오일 코알라주유소', '대성석유(주)길동주유소', '(주)소모 성내주유소', 'sk해뜨는주유소', '지에스칼텍스㈜ 동서울주유소', '천호현대주유소', '광성주유소', '수유동주유소', '㈜지에스이앤알 미아주유소', '덕릉로주유소', '(주)석산에너지', '(주)서울에너지 시민주유소', '씨앤에스유통㈜ 미아셀프주유소', 'HD현대오일뱅크㈜직영 번동셀프주유소', '북서울고속주유소', '세원주유소', '대성석유(주)신광주유소', 'SK에너지(주) 매일주유소', '에덴주유소', '화곡역주유소', '이케이에너지(주) 강서주유소', '목화주유소', '(주)뉴신정주유소', '유턴주유소', '(주)타이거오일 방화주유

In [20]:
len(name_list), len(address_list), len(brand_list), len(gasolin_list), len(disel_list), len(cwsh_yn_list), len(lpg_yn_list), len(maint_yn_list), len(cvs_yn_list), len(sel24_yn_list)

(440, 440, 440, 440, 440, 440, 440, 440, 440, 440)

In [34]:
# 리스트 저장
import pickle
gas_station_dict = {
    'name_list' : name_list,
    'address_list' : address_list,
    'brand_list' : brand_list,
    'gasolin_list' : gasolin_list,
    'disel_list' : disel_list,
    'cwsh_yn_list' : cwsh_yn_list,
    'lpg_yn_list' : lpg_yn_list,
    'maint_yn_list' : maint_yn_list,
    'cvs_yn_list' : cvs_yn_list,
    'sel24_yn_list' : sel24_yn_list
}

In [35]:
gas_station_dict['name_list']

['(주)보성 세곡주유소',
 'HD현대오일뱅크㈜직영 산성셀프주유소',
 '방죽주유소',
 '자곡셀프주유소',
 'HD현대오일뱅크㈜직영 도곡셀프주유소',
 '오일프러스 셀프',
 'HD현대오일뱅크㈜직영 유진주유소',
 'SK서광주유소',
 '극동유화㈜ 개나리주유소',
 '(주)선진도곡주유소',
 '(주)중앙에너비스 수서지점',
 '일원주유소',
 '지에스칼텍스㈜에너지플러스허브GS타워',
 '대교셀프주유소',
 '삼성동주유소',
 'SK에너지㈜ 진달래주유소',
 'SK에너지(주) 매봉주유소',
 '(유)동하석유 힐탑셀프주유소',
 '대동석유 압구정주유소',
 '에스제이에너지산업(주) 개포주유소',
 'HD현대오일뱅크㈜직영 신사현대주유소',
 '지에스칼텍스㈜ 은마주유소',
 'HD현대오일뱅크㈜직영 한양주유소',
 '지에스칼텍스(주)학여울주유소',
 '지에스칼텍스㈜ 삼성로주유소',
 '대성석유(주)동호주유소',
 '대청주유소',
 '명품주유소',
 '(주)소모 쎈트럴주유소',
 '㈜새서울석유 새서울주유소',
 '갤러리아주유소',
 'SK논현주유소',
 '(주)제이제이네트웍스 제이제이주유소',
 '(주)만정에너지 삼보주유소',
 '재건에너지 재정제2주유소 고속셀프지점',
 '구천면주유소',
 '방아다리주유소',
 '(주)삼표에너지 고덕주유소',
 '주)지유에너지직영 오렌지주유소',
 '명일주유소',
 '(주)소모 신월주유소',
 '(주)퍼스트오일 코알라주유소',
 '대성석유(주)길동주유소',
 '(주)소모 성내주유소',
 'sk해뜨는주유소',
 '지에스칼텍스㈜ 동서울주유소',
 '천호현대주유소',
 '광성주유소',
 '수유동주유소',
 '㈜지에스이앤알 미아주유소',
 '덕릉로주유소',
 '(주)석산에너지',
 '(주)서울에너지 시민주유소',
 '씨앤에스유통㈜ 미아셀프주유소',
 'HD현대오일뱅크㈜직영 번동셀프주유소',
 '북서울고속주유소',
 '세원주유소',
 '대성석유(주)신광주유소',
 'SK에너지(주) 매일주유소',
 '에덴주유소',
 '화곡역주유소',
 '이케

In [36]:
gas_station_dict['brand_list']

['SK에너지',
 'HD현대오일뱅크',
 'GS칼텍스',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'S-OIL',
 'S-OIL',
 'SK에너지',
 'S-OIL',
 'GS칼텍스',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'SK에너지',
 'SK에너지',
 'SK에너지',
 'GS칼텍스',
 'HD현대오일뱅크',
 'GS칼텍스',
 'HD현대오일뱅크',
 'GS칼텍스',
 'GS칼텍스',
 'GS칼텍스',
 'SK에너지',
 'S-OIL',
 'GS칼텍스',
 'HD현대오일뱅크',
 'SK에너지',
 'SK에너지',
 'HD현대오일뱅크',
 'GS칼텍스',
 'HD현대오일뱅크',
 'HD현대오일뱅크',
 'SK에너지',
 'GS칼텍스',
 'SK에너지',
 'SK에너지',
 'GS칼텍스',
 'S-OIL',
 'GS칼텍스',
 'GS칼텍스',
 'SK에너지',
 'GS칼텍스',
 'HD현대오일뱅크',
 'S-OIL',
 'GS칼텍스',
 'GS칼텍스',
 'S-OIL',
 'HD현대오일뱅크',
 'HD현대오일뱅크',
 'S-OIL',
 'HD현대오일뱅크',
 'SK에너지',
 'SK에너지',
 'GS칼텍스',
 'SK에너지',
 'SK에너지',
 '알뜰주유소',
 'HD현대오일뱅크',
 '알뜰주유소',
 '알뜰주유소',
 '알뜰주유소',
 'SK에너지',
 'GS칼텍스',
 'SK에너지',
 'S-OIL',
 'HD현대오일뱅크',
 'HD현대오일뱅크',
 'S-OIL',
 'SK에너지',
 'S-OIL',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'SK에너지',
 'GS칼텍스',
 'SK에너지',
 'GS칼텍스',
 'GS칼텍스',
 'GS칼텍스',
 'GS칼텍스',
 'GS칼텍스',
 'SK에너지',
 'S-OIL',
 'SK에너지',
 'S-OIL',
 'S-OIL',
 'SK에너지',
 'HD현대오일뱅크',


In [37]:
gas_station_dict['maint_yn_list']

['https://www.opinet.co.kr/images/user/gis/oil_station_service1_03_off.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03_off.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03_off.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03_off.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03_off.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03_off.gif',
 'https://www.opinet.co.kr/images/user/gis/oil_station_service1_03.gif',
 'https://www.opinet.co.kr/

In [38]:
# Save raw data
with open('../data/gas_station_dict.pickle', 'wb') as f:
    pickle.dump(gas_station_dict, f, pickle.HIGHEST_PROTOCOL)

In [39]:
# load
with open('../data/gas_station_dict.pickle', 'rb') as f:
    gas_station_dict_loaded = pickle.load(f)

In [40]:
gas_station_dict_loaded['brand_list']

['SK에너지',
 'HD현대오일뱅크',
 'GS칼텍스',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'S-OIL',
 'S-OIL',
 'SK에너지',
 'S-OIL',
 'GS칼텍스',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'SK에너지',
 'SK에너지',
 'SK에너지',
 'GS칼텍스',
 'HD현대오일뱅크',
 'GS칼텍스',
 'HD현대오일뱅크',
 'GS칼텍스',
 'GS칼텍스',
 'GS칼텍스',
 'SK에너지',
 'S-OIL',
 'GS칼텍스',
 'HD현대오일뱅크',
 'SK에너지',
 'SK에너지',
 'HD현대오일뱅크',
 'GS칼텍스',
 'HD현대오일뱅크',
 'HD현대오일뱅크',
 'SK에너지',
 'GS칼텍스',
 'SK에너지',
 'SK에너지',
 'GS칼텍스',
 'S-OIL',
 'GS칼텍스',
 'GS칼텍스',
 'SK에너지',
 'GS칼텍스',
 'HD현대오일뱅크',
 'S-OIL',
 'GS칼텍스',
 'GS칼텍스',
 'S-OIL',
 'HD현대오일뱅크',
 'HD현대오일뱅크',
 'S-OIL',
 'HD현대오일뱅크',
 'SK에너지',
 'SK에너지',
 'GS칼텍스',
 'SK에너지',
 'SK에너지',
 '알뜰주유소',
 'HD현대오일뱅크',
 '알뜰주유소',
 '알뜰주유소',
 '알뜰주유소',
 'SK에너지',
 'GS칼텍스',
 'SK에너지',
 'S-OIL',
 'HD현대오일뱅크',
 'HD현대오일뱅크',
 'S-OIL',
 'SK에너지',
 'S-OIL',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'HD현대오일뱅크',
 'SK에너지',
 'SK에너지',
 'GS칼텍스',
 'SK에너지',
 'GS칼텍스',
 'GS칼텍스',
 'GS칼텍스',
 'GS칼텍스',
 'GS칼텍스',
 'SK에너지',
 'S-OIL',
 'SK에너지',
 'S-OIL',
 'S-OIL',
 'SK에너지',
 'HD현대오일뱅크',


In [41]:
driver.close()